In [1]:
!pip install matplotlib
import matplotlib.pyplot as plt

#### Exercice 3

In [8]:
%%file exam3_hanan.py
from mpi4py import MPI
import numpy as np

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()

def distribute_data(matrice, size):
    n = matrice.shape[0]
    part_size = n // size
    reste = n % size

    data = []
    start_row = 0
    for i in range(size):
        end_row = start_row + part_size + (1 if i < reste else 0)
        data.append(matrice[start_row:end_row, :])
        start_row = end_row

    return data

if RANK == 0:
    # initialiser la taille de la matrice
    n = 8

    # déclaration de la matrice
    matrice = np.zeros((n, n))

    # Initialiser la diagonale de la matrice de 1 à n
    for i in range(n):
        matrice[i, i] = i + 1

    # Distribuer la matrice en parties 
    data = distribute_data(matrice, SIZE)
else:
    data = None

# Envoyer les parties de la matrice aux autres processus (les sous matrice)
local_matrice = COMM.scatter(data, root=0)

# Calculer la somme locale des éléments
local_sum = np.sum(local_matrice)

# Rassembler les résultats dans le processus 0
all_sums = COMM.gather(local_sum, root=0)

# Calculer la trace globale
if RANK == 0:
    global_sum = sum(all_sums)
    print("La trace de la matrice est  :", global_sum)




Overwriting exam3_hanan.py


In [9]:
!mpirun -n 4 python3 exam3_hanan.py

La trace de la matrice est  : 36.0


In [7]:
'''
from mpi4py import MPI
import numpy as np

COMM = MPI.COMM_WORLD
SIZE = COMM.Get_size()
RANK = COMM.Get_rank()
def f(x): 
        return 0

#long = 2
#larg= 2
nx = 101
ny= 101
#les pas de mayge
dx =2/(Nx-1)
dy = 2/(Ny-1)
#conndition initialle 
x = np.zeros((nx, ny))
y= np.zeros((nx, ny))
u = np.zeros((nx, ny))
for i in range(nx):
    for j in range(ny):
        x[i,j]= i*dx
        y[i,j]= j*dy
        u[i,j]= 0
temps = 0
a1 = 1
a2 = 1
D = 0.25
tfinal = 51
CFL =0.5
dt1 = CFL*min(dx, dy)/max(abs(a1),abs(a2))
dt2 = CFL*min(dx**2,dy**2)/(2*D)
dt = min(dt1,dt2)
lamda1= dt*a1/dx
lamda2= dt*a2/dy
beta1 = dt*D/dx**2
beta2 = dt*D/dy**2
unew = np.zeros((Nx,Ny))
while(temps< tfinal):
    for i in range(1,nx-1):
        for j in range(1,ny-1):
            unew[i,j]= u[i,j]-lamda1*(u[i,j]-u[i-1,j])\
                -lamda2*(u[i,j]-u[i,j-1])\
                +beta1*(u[i-1,j]-2*u[i,j]+u[i+1,j])\
                +beta2*(u[i,j-1]-2*u[i,j]+u[i,j+1])
                
    unew[:,0] = unew[:,1]
    unew[:,ny-1] = unew[:,ny-2]
    unew[nx-1,:]= unew[nx-2,:]
    for j in range(ny):
         if (40<= y[0,j]<=60):
             unew[0,j]=1
         else:
            unew[0,j]=0
    temps += dt   
    u = unew.copy()
    '''

Overwriting exam3_hanan.py


In [ ]:
import numpy as np 
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D
import time
def solve_2d_diff(u,un,nt,dt,dx,dy,nu):
    row, col =u.shape
    ### Assign initial conditions
    # set hat function I.C. : u(0.5<=x<=1 && 0.5<=y<=1) is 2
    u[int(0.5/dy): int(1/ dy +1), int(0.5/dx):int(1/dx+1)]=2


    #fill the update of u 
    return u
nt= 51
nx=101
ny=101
nu=0.05
dx= 2/(nx-1)
dy=2/(ny-1)
sigma = 0.25
dt= sigma*dx*dy/nu
x = np.linspace(0,2,nx)
y=np.linspace(0,2,ny)
u = np.ones((ny,nx)) #create a 1xn vector of 1's
un= np.ones(ny,nx)
solve_2d_diff(u,un,dt,dx,dy,nu)
fig = pyplot.figure(figsize=(7,5), dpi=100)
ax= fig.gca(projection='3d')
X,Y=np.meshgrid(x,y)
surf = ax.plot_surface(X,Y,u,cmap=cm.viridis)


In [10]:
%%file exam4_hanan.py
import numpy as np 
from matplotlib import pyplot, cm
from mpl_toolkits.mplot3d import Axes3D

def solve_2d_diff(u, un, nt, dt, dx, dy, nu):
    row, col = u.shape
    # Assigner les conditions initiales
    # Définir une fonction chapeau I.C. : u(0.5<=x<=1 && 0.5<=y<=1) vaut 2
    u[int(0.5 / dy): int(1 / dy + 1), int(0.5 / dx):int(1 / dx + 1)] = 2

    # Boucle temporelle
    for n in range(nt + 1):
        un = u.copy()
        for i in range(1, row - 1):
            for j in range(1, col - 1):
                u[i, j] = un[i, j] + nu * dt / dx**2 * (un[i + 1, j] - 2 * un[i, j] + un[i - 1, j]) \
                                      + nu * dt / dy**2 * (un[i, j + 1] - 2 * un[i, j] + un[i, j - 1])

        # Appliquer les conditions aux limites
        u[:, 0] = 1  # Bord gauche
        u[:, -1] = 1  # Bord droit
        u[0, :] = 1  # Bord inférieur
        u[-1, :] = 1  # Bord supérieur

    return u

# Paramètres du problème
nt = 51
nx = 101
ny = 101
nu = 0.05
dx = 2 / (nx - 1)
dy = 2 / (ny - 1)
sigma = 0.25
dt = sigma * dx * dy / nu

x = np.linspace(0, 2, nx)
y = np.linspace(0, 2, ny)
u = np.ones((ny, nx))  # Créer une matrice de dimensions nx par ny remplie de 1
un = np.ones((ny, nx))  # Matrice temporaire pour les anciennes valeurs de u

# Résoudre l'équation de diffusion
u = solve_2d_diff(u, un, nt, dt, dx, dy, nu)

# Afficher la solution
fig = pyplot.figure(figsize=(7, 5), dpi=100)
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(x, y)
surf = ax.plot_surface(X, Y, u, cmap=cm.viridis)
pyplot.show()


Writing exam4_hanan.py


In [11]:
!mpirun -n 4 python3 exam4_hanan.py

Traceback (most recent call last):
  File "/home/asus/S2/calcul paralléle/exam4_hanan.py", line 47, in <module>
Traceback (most recent call last):
  File "/home/asus/S2/calcul paralléle/exam4_hanan.py", line 47, in <module>
    ax = fig.gca(projection='3d')
    ax = fig.gca(projection='3d')
         ^^^^^^^^^^^^         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^
TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'
TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'
Traceback (most recent call last):
  File "/home/asus/S2/calcul paralléle/exam4_hanan.py", line 47, in <module>
    ax = fig.gca(projection='3d')
         ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: FigureBase.gca() got an unexpected keyword argument 'projection'
Traceback (most recent call last):
  File "/home/asus/S2/calcul paralléle/exam4_hanan.py", line 47, in <module>
    ax = fig.gca(projection='3d')
         ^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: FigureBase.gca() got an unexpected keywor